In [1]:
!pip install MRJob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 6.3 MB/s eta 0:00:00


In [81]:
%%writefile data.txt
log 1: Uid=1, Errors =,a, b, a, j, e, c, e, k ,j
log 2: Uid=2, Errors =,a, d,
log 3: Uid=1, Errors =,a, b, e,
log 4: Uid=3, Errors =,a, a, a, a, a, c, b

Overwriting data.txt


In [82]:
%%writefile Q1file.py

from mrjob.job import MRJob

class Q1(MRJob):

    def mapper(self, _, line):
        try:
            parts = line.split('Uid=')
            uid_part = parts[1].split(',')[0].strip()
            key = f"Uid={uid_part}"

            errors_part = parts[1].split('Errors =')[1]
            errors = errors_part.split(',')
            dic = {}
            for error in errors:
                error = error.strip()
                if error:
                    if error not in dic:
                        dic[error] = 1
                    else:
                        dic[error] += 1

            yield key, dic
        except IndexError:
            return

    def reducer(self, key, values):
        combined_dic = {}
        for value_dic in values:
            for error, count in value_dic.items():
                if error not in combined_dic:
                    combined_dic[error] = count
                else:
                    combined_dic[error] += count

        if combined_dic:
            max_count = max(combined_dic.values())
            max_keys = [k for k, v in combined_dic.items() if v == max_count]
            yield key, max_keys

if __name__ == '__main__':
    Q1.run()


Overwriting Q1file.py


In [83]:
!python Q1file.py  data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/Q1file.root.20240517.175959.102631
Running step 1 of 1...
job output is in /tmp/Q1file.root.20240517.175959.102631/output
Streaming final output from /tmp/Q1file.root.20240517.175959.102631/output...
"Uid=2"	["a", "d"]
"Uid=3"	["a"]
"Uid=1"	["a", "e"]
Removing temp directory /tmp/Q1file.root.20240517.175959.102631...


In [32]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=94991aff8f311b64c1c12020bb88b6f8b507429a53995f257fdcd972e64c01e7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [36]:
from pyspark import SparkContext,SparkConf
conf=SparkConf().setAppName('A2').setMaster("local[*]")
sc=SparkContext.getOrCreate(conf=conf)

In [107]:
%%writefile data2.txt
log 1: Uid=1,Errors =a,b,a,j,e,c,e,k,j
log 2: Uid=2,Errors =a,d
log 3: Uid=1,Errors =a,b,e
log 4: Uid=3,Errors =a,a,a,a,a,c,b

Overwriting data2.txt


In [108]:
def count(line):
  key=line[0]
  valueslst=line[1]
  dic={}
  for x in valueslst:
    if x not in dic:
      dic[x]=1
    else:
      dic[x]+=1

  max_count = max(dic.values())
  max_keys = [k for k, v in dic.items() if v == max_count]
  return (key, max_keys)


rdd=sc.textFile('data2.txt')
mapped_rdd=rdd.map(lambda x :((x[7:12]),(x[21::]).split(',')))


grouped_rdd=mapped_rdd.reduceByKey(lambda x,y :x+y)
max_rdd=grouped_rdd.map(count)
for x in max_rdd.collect():
 print(x)


('Uid=2', ['a', 'd'])
('Uid=1', ['a', 'e'])
('Uid=3', ['a'])


In [109]:
%%writefile data3.txt
A->B C D
B->A D
C->A
D->B C


Writing data3.txt


In [206]:
dic={}
def custom(line):
    global dic
    key = line[0]
    value = line[1].split()  # Split the value string into a list
    if key in dic.keys():
        dic[key].extend(value)
    else:
        dic[key] = value





def counter(line):
  global count,dic
  key=line[0]
  value=line[1].split()
  count=0
  for reciver in value:
    if key not in dic[reciver]:
      count=1
      return ("Number of users who sent a message but didn't receive a reply:",count)







rdd=sc.textFile('data3.txt')
mapped_rdd=rdd.map(lambda x :(x.split('->')))
for x in mapped_rdd.collect():
  custom(x)

final_rdd=mapped_rdd.map(counter)
final_rdd=final_rdd.filter(lambda x : x is not None)
final_ans=final_rdd.reduceByKey(lambda x,y: x+y)
final_ans.collect()

[("Number of users who sent a message but didn't receive a reply:", 2)]

In [210]:
rdd=sc.textFile('data3.txt')
ans=rdd.map(lambda x :(x.split('->'))).count()

print(ans)

4


In [232]:
rdd=sc.textFile('data3.txt')
ans=rdd.map(lambda x :(x.split('->')))
ans1=ans.map(lambda x :((len((x[0]+' '+x[1]).split()))))
print(ans1.reduce(lambda x,y :x+y))

12
